In [2]:
import pandas as pd
import pybedtools as bt
from glob import glob
import numpy as np
import pybedtools as bt
import pyBigWig as bw
import os
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 100

import warnings
warnings.filterwarnings("ignore")

os.chdir('/fast/work/groups/ag_kircher/ReMM/MA_Lusi/Snakemake')

In [2]:
import random
'snakemake -n output/predictions/hg38/SNVs.hg38.predictions.txt_'+str(random.sample(range(10000, 99999), 50)).replace(' ','').replace('[','{').replace(']','}')


'snakemake -n output/predictions/hg38/SNVs.hg38.predictions.txt_{79831,29779,19259,15152,96584,26373,56705,70836,63553,80755,78121,22797,92565,14842,54865,19484,88549,33145,39400,64327,49468,21150,30844,38251,93902,79273,95330,20673,53504,54216,66408,78518,54871,93623,37164,98729,18102,43164,77581,83137,85174,57134,84389,88901,61887,19171,61435,91673,30544,20311}'

### Funtions

In [2]:
def getData(r):
    di = {'chr5':'PRDM9','chr11':'HBB','chr14':'DLK1'}

    to_replace = {'priPhCons':'priPhastCons',
        'mamPhCons':'mamPhastCons',
        'verPhCons':'verPhastCons',
        'priPhyloP':'priPhyloP',
        'mamPhyloP':'mamPhyloP',
        'verPhyloP':'verPhyloP',
        'EncodeH3K4me1-max'   :'EncH3K4Me1',
        'EncodeH3K4me3-max': 'EncH3K4Me3',
        'EncodeH3K27ac-max' :   'EncH3K27Ac',
        'Pos':'POSITION',
        'GerpRS': 'GerpRS',
         'GerpRSpval':'GerpRSpv',
        'EncOCDNaseSig':'DnaseClusteredScore',
         'EncodeDNase-max':'DnaseClusteredScore',
        'EncodeDNase-sum':'DnaseClusteredHyp',
        'EncOCDNasePVal':'DnaseClusteredHyp',
                 'GC':'GCContent',
                 'CpG':'CpGperCpG',
                 'GCContent':'GCContent',
                 'CpGperCpG':'CpGperCpG'}


    for chrom in list(di.keys())[0:1]:
        chrom = r
        c = di[chrom]
        #print(chrom)
        cadd19_ = pd.read_csv('/data/groups/ag_kircher/ReMM/MA_Lusi/CADD_feature_comparison/'+c+'_hg19.tsv.gz',sep = '\t') 
        cadd38_ = pd.read_csv('/data/groups/ag_kircher/ReMM/MA_Lusi/CADD_feature_comparison/'+c+'_hg38.tsv.gz',sep = '\t') 


        cols = list(pd.Series(to_replace).unique())
        cadd38_.columns = pd.Series(cadd38_.columns).replace(to_replace)
        cadd38 = cadd38_.fillna(0).loc[:,cols]
        cadd19_.columns = pd.Series(cadd19_.columns).replace(to_replace)
        cadd19 = cadd19_.fillna(0).loc[:,cols]

        cols = hg38.columns.to_list()
        df19 = pd.read_csv('validation/'+chrom+'.features19.txt.gz',sep = '\t')
        df19.columns = df19.columns.str.replace('46way','')
        df19 = df19.loc[:,cols]
        df19 = df19.fillna(0).round(3)

        df38 = pd.read_csv('validation/'+chrom+'.features38.txt.gz',sep = '\t')
        df38.columns = df38.columns.str.replace('Fantom','fantom')
        df38 = df38.loc[:,cols]
        df38 = df38.fillna(0).round(3)

       # print(chrom)
        modes = ['EncH3K27Ac','EncH3K4Me3','EncH3K4Me1']
        modes = ['priPhastCons','priPhyloP','verPhastCons','verPhyloP','mamPhastCons','mamPhyloP']


        for mode in modes:
         #   print(mode)
            #print(pd.DataFrame([cadd19[mode],df19[mode],hg19[mode]]).T.corr())
            #print(pd.DataFrame([cadd38[mode],df38[mode],hg38[mode]]).T.corr())

        #print("19: cadd und df \n\n",cadd19.corrwith(df19))
        #print("19: cadd und df  \n\n",cadd38.corrwith(df38))
        #print("19: cadd und hg  \n\n",cadd19.corrwith(hg19))
        #print("38: cadd und hg  \n\n",cadd38.corrwith(hg38))

        #print(cadd19.corrwith(cadd38))ich m
        #print(df19.corrwith(df38))
        return cadd19,cadd38, df19,df38



In [4]:
#import json


#with open('config/features_config.json') as f:
#    data = json.load(f)

In [7]:
df = pd.DataFrame()

In [3]:
class Validate:
    def __init__(self,b ='hg38',r = 2):
    
        regions = [{'hg19':['chr5',23464354,23570942],'hg38':['chr5',23464245,23570833]},
                    {'hg19':['chr11',5167199,5327798],'hg38':['chr11',5145969,5306568]},
                    {'hg19':['chr14',101118632,101242576],'hg38':['chr14',100652295,100776239]},
                   {'hg19':['chr14',101118632,201242576],'hg38':['chr14',100652295,200776239]}]
        self.di = regions[r]
        self.b = b
        self.start = self.di[self.b][1]-1
        self.end = self.di[self.b][2]
        self.chr = self.di[self.b][0]
        
    def get_bed(self):
        
        # die Positionen in den Regionen sind 1-basiert (wegen featureVcfHg19)
        # eine BED Datei erstellen: Start und Endposition -1

        bed = pd.DataFrame()
        #print(start,end)
        s = self.start 
        e = self.end 
        
        bed['START'] =  range(s,e)
        bed['END'] =  range(s+1,e+1)
        bed['CHR'] = self.chr
        self.bed = bed[['CHR','START','END']]
        return self.bed
    
    def getEncode(self):
        # die Originaldateien für hg19 sind heruntergeladen (Link in der feature_config)
        # 1-basiret, die Werte sind mit pyBigWig extrahiert, der Maximum von allen Zelllinien ist berechnet
        self.mods = ['EncH3K27Ac','EncH3K4Me3','EncH3K4Me1']
        #self.mods = ['EncH3K4Me3']

        hg = pd.DataFrame()
        for mod in self.mods:
            vv = []
            files = glob('input/features/'+self.b+'/'+mod+'/*.bigWig')
            for file in files:
                #print(file)
                self.file = file
                bb = bw.open(self.file )
                v = bb.values(self.di[self.b][0], self.start, self.end) 
                vv.append(v)
            df = pd.DataFrame(vv).T
            hg[mod] = df.max(axis = 1)
        print(hg.shape)
        return hg, df
    
    
    def addID(self,df):
        
        # eine ID hinzufügen wenn die Featuer Bed sie nicht hat sodass Ausgabe für alle Features gleich ist
        df['id'] = df[0]+':'+df[1].astype(str)+':'+df[2].astype(str)
        df = df[[0,1,2,'id']+df.columns[3:-1].to_list()]
        return df
    
    
    def intersect_with_bed(self,featureBed):
        
        # Features im BED Format werden mit pybedtools extrahiert, dafür wird Intersection zwischen
        # der Feature Datei und der BED Dataframe vom getBed berechnet. Beide Dateien werden zuerst zum BedTool konvertiert
        # zusätzlich ein paar column name manipualtions 
        self.featureBed = featureBed
        cols = self.featureBed.columns.astype(str)

        self.bedTool = bt.BedTool.from_dataframe(self.get_bed())
        self.featureBed = bt.BedTool.from_dataframe(self.featureBed)
        featureIntersect = self.bedTool.intersect(self.featureBed, loj = True)
        self.featureIntersect = featureIntersect.to_dataframe(disable_auto_names=True,names=cols)
        
        self.featureIntersect.index.names =['chr','start','end']
        self.featureIntersect = self.featureIntersect.reset_index()
        #c = pd.Series(self.featureIntersect.columns)
        #self.featureIntersect.columns = pd.to_numeric(c, errors='coerce').fillna(c)
        
        return self.featureIntersect.replace(-1,np.nan)
    
    def get_fantom(self):
         
        # getFantom gibt eine DF mit Duplikaten zurück.
        # habe sie erstmal gedropped
        perm = pd.read_csv('input/features/'+self.b+'/Fantom5Perm/Fantom5Perm.all.bed.gz',sep = '\t', header = None,usecols= [0,1,2,3,4])
        robust = pd.read_csv('input/features/'+self.b+'/Fantom5Robust/Fantom5Robust.all.bed.gz',sep = '\t', header = None,usecols= [0,1,2,3,4])
        
        
        self.perm_df = self.intersect_with_bed(perm)
        self.robust_df = self.intersect_with_bed(robust)
        
        self.perm_df = self.perm_df.groupby('start')['4'].mean().reset_index(drop=True)
        self.robust_df = self.robust_df.groupby('start')['4'].mean().reset_index(drop=True)
        
        df = pd.DataFrame()
        df['fantom5Perm'] = self.perm_df#['4']
        df['fantom5Robust']= self.robust_df#['4']
        
        return df.replace(-1,np.nan)
    


    
    def getDnase(self):
        
        # Dnase Dateien sind auch im BED Format
        dnase = pd.read_csv('input/features/'+self.b+'/DnaseClusteredHyp/DnaseClusteredHyp.all.bed.gz', sep = '\t', header = None,usecols = [0,1,2,3,4])
        self.dnase_df = self.addID(dnase)

        self.dnase_df = self.intersect_with_bed(self.dnase_df)
        
        
        return self.dnase_df.iloc[:,-2:].apply(lambda x: x.str.replace('.','0')).astype(float)
    
    def get_conservation(self):
        
        # für hg19 sind die conservation scores für die 3 Chromosome heruntergeladen
        # Dateien für beide Builds sind von wigFix to BigWig convertiert
        # 1-basiert
        hg = pd.DataFrame()
        self.consScores = ['priPhastCons','priPhyloP','verPhastCons','verPhyloP','mamPhastCons','mamPhyloP']
        for score in self.consScores:
           # print(score)
            if ('pri' in score) and (self.b=='hg38'):
                st = 'input/features/validation/hg38/'+score+'/'+'*.bw'
                #print(st)

                file = glob(st)
            else: 
                st = 'input/features/validation/hg38/'+score+'/'+self.di[self.b][0]+'*.bw'
                file = glob(st) #chr11_phastCons100way.bw
                #print(st)

            if self.b=='hg19':
                st = 'input/features/hg19/'+score+'/'+self.di[self.b][0]+'_'+'*.bw'
                #print(st)

                file = glob(st)
            if len(file)>1:
                print('More than one BW!')
                print(file)
            bb = bw.open(file[0])
            v = bb.values(self.chr, self.start, self.end) 
            hg[score] = v
        #print(hg.shape)
        return hg
    
    def getGCIslands(self):
        
        gc = pd.read_csv('input/features/'+self.b+'/cpgIslandExt/cpgIslandExt.all.bed.gz', sep = '\t', header = None,usecols =[0,1,2,7,8,9])
        self.gc = self.addID(gc)
        #print(self.gc.head())
        self.gc_df = self.intersect_with_bed(self.gc)[['7','8','9']]
        #print(self.gc_df.head())

        self.gc_df.columns = self.gc_df.columns.astype(int)
        return self.gc_df[[7,8,9]].apply(lambda x: x.str.replace('.','0')).astype(float)
    
    def getCGContent(self):
        gc =bt.BedTool('input/features/hg38/GCContent/GCContent.all.bed.gz')
    
    
    
    def getGerpRS(self):
        # assume that Gerp is 0-based/ no info in description on the webpage
        gerp = pd.read_csv('input/features/'+self.b+'/GerpRS/GerpRS.all.bed.gz', sep='\t', header = None)
        self.gerp_df = self.addID(gerp)
        self.gerp_df = self.intersect_with_bed(self.gerp_df)
        return self.gerp_df.iloc[:,-2:].apply(lambda x: x.str.replace('.','0')).astype(float)

    def getSTVar(self,feature, file):    
        #print('input/features/'+self.b+'/'+feature+'/'+file +'.bed.gz')
        self.stv = pd.read_csv('input/features/'+self.b+'/'+feature+'/'+file +'.bed.gz',sep = '\t',header= None,usecols = [0,1,2,3])
        self.stv_df = self.addID(self.stv )
        self.stv_df = self.intersect_with_bed(self.stv_df)
        #display(self.stv_df)
        return self.stv_df['3']
    
    def getDGVCount(self,):
        alle = pd.DataFrame()
        st ='input/features/'+self.b+'/DGVCount/DGVCount.'+self.chr+'.bed.gz'
        file = glob(st)
        #print(st)
        dgv = pd.read_csv(file[0],compression='gzip',sep = '\t',header = None)
        self.dgv = dgv.sort_values(dgv.columns.to_list())[[0,1,2,3]]

        self.dgv_df = self.addID(self.dgv)
        self.dgv_df = self.intersect_with_bed(self.dgv_df)
        
        return self.dgv_df['3']
        
    def getAllSTVar(self):
        
        fdi={'ISCApath':'ISCApath.nstd37_nstd45_nstd75','dbVARCount':'dbVARCount.all'}
        #fdi={'DGVCount':'DGVCount.all'}

        df = pd.DataFrame()
        for feature in fdi.keys():
            #print(feature)
            file = fdi[feature]
            df[feature]=self.getSTVar(feature,file)
        return df[fdi.keys()]
    
    
    
    def getAll(self):
        
        df = pd.DataFrame()
        df['POSITION'] = range(self.di[self.b][1],self.di[self.b][2]+1)
        df[self.mods] = self.getEncode()[0]
        df[['fantom5Perm','fantom5Robust']] = self.get_fantom()
        df[self.consScores] = self.get_conservation()
        df[['DnaseClusteredHyp','DnaseClusteredScore']]=self.getDnase()
        df[['CpGperCpG','CpGperGC','CpGobsExp']] = self.getGCIslands()
        df[['GerpRS','GerpRSpv']] = self.getGerpRS()
        df[['ISCApath','dbVARCount']] = self.getAllSTVar()
        df[['DGVCount']] = self.getDGVCount()

        
        
        return df.fillna(0).astype(float).round(3)
        





### All the features are comment 'out' except for variants/ to change - comment 'in' in functions (last lines)

In [89]:
# r is one of the intervals (0,1,2)
df = pd.DataFrame()
df['col']=col
df = df.set_index('col')
for r in [0,1,2]:
   
    x38 = Validate(b = 'hg38', r=r)
    hg38 = x38.getAll()
    print('----------------')
    x19 = Validate(b = 'hg19',r=r)
    hg19 = x19.getAll()
    d = hg19.corrwith(hg38)
    d.name = r
    df = df.join(d)


(106589, 3)
priPhastCons
priPhyloP
verPhastCons
verPhyloP
mamPhastCons
mamPhyloP
----------------
(106589, 3)
priPhastCons
priPhyloP
verPhastCons
verPhyloP
mamPhastCons
mamPhyloP
(160600, 3)
priPhastCons
priPhyloP
verPhastCons
verPhyloP
mamPhastCons
mamPhyloP
----------------
(160600, 3)
priPhastCons
priPhyloP
verPhastCons
verPhyloP
mamPhastCons
mamPhyloP
(123945, 3)
priPhastCons
priPhyloP
verPhastCons
verPhyloP
mamPhastCons
mamPhyloP
----------------
(123945, 3)
priPhastCons
priPhyloP
verPhastCons
verPhyloP
mamPhastCons
mamPhyloP


### Identical regions + Random

In [14]:
cor = []
l = []
for c in ['chr5','chr11','chr14']:

    d1 = pd.read_csv('output/predictions/'+c+'/SNVs.'+c+'.predictions.txt',sep = '\t',header = None )[0]
    d2 = pd.read_csv('output/predictions/'+c+'/SNVs.'+c+'.remm.txt',sep = '\t',header = None )[2]
    cor.append(pd.DataFrame([d1,d2]).T.corr().iloc[0,1])
    l.append(d2.shape[0])
    
cor.append(0.602342)
c = pd.read_csv('output/predictions/random/SNVs.random.predictions.txt',sep= '\t',header = None).shape[0]
c2 = pd.read_csv('output/predictions/random/hg19random.remm.txt.gz',sep= '\t',header = None)#.shape[0]


l.append(c)

d = pd.DataFrame([l,cor])
d.columns = ['DLK1','HBB','PRDM9','Random']
d.index = ['Length','Correlation']
#d.to_latex('figures/lc.tex', index = True,header = True)

,0,1
DLK1,106589.0,0.575921
HBB,160600.0,0.664163
PRDM9,123945.0,0.639591
Random,119533.0,0.602342


In [104]:
## tree identical regions + radom
col = ['CpGperGC', 'CpGperCpG', 'CpGobsExp','GCContent','EncH3K27Ac', 'EncH3K4Me1',
       'EncH3K4Me3', 'verPhyloP', 'mamPhyloP',
       'priPhyloP','priPhastCons', 'mamPhastCons','verPhastCons', 'DnaseClusteredHyp', 'DnaseClusteredScore', 
         'Fantom5Perm','Fantom5Robust','GerpRS', 'GerpRSpv', 'rareVar', 'commonVar', 'fracRareCommon',
        'numTFBSConserved',
       'ISCApath', 'DGVCount', 'dbVARCount']

df = pd.DataFrame()
df['col']=col
df = df.set_index('col')

for c in ['chr5','chr11','chr14']:
    d = pd.read_csv('validation/'+c+'.features19.txt.gz',sep = '\t')
    d.columns = d.columns.str.replace('46way','')
    d.columns = d.columns.str.replace('fantom','Fantom')

    d1 = pd.read_csv('output/features/annotated/'+c+'/SNVs.'+c+'.combined.txt.gz',sep = '\t')
    
    j = d.corrwith(d1)
    j.name = c
    df = df.join(j)
    
    
file = 'output/predictions/random/hg19random.features.vcf.gz'
d = pd.read_csv(file,sep = '\t',skiprows = 0)
#d[d[0].str.len()<6].to_csv(file,sep = '\t',index =False,header = False)
#d.head(100)

d.columns = d.columns.str.replace('46way','')
d.columns = d.columns.str.replace('fantom','Fantom')


file = 'output/predictions/random/hg38random.features.vcf.gz'
d1 = pd.read_csv(file,sep = '\t',skiprows = 0)
d1['ID']=d1['CHR']+':'+d1['POSITION'].astype(str)

d['ID']=d['ID'].str.split(':',expand = True)[0]+':'+d['ID'].str.split(':',expand = True)[1]


dd = d.join(d1,lsuffix = '_ReMM',rsuffix='_parSMURF',how = 'inner')


d19 = dd.loc[:,dd.columns.str.contains('ReMM')]
d19.columns = d19.columns.str.replace('_ReMM','')
d38 = dd.loc[:,dd.columns.str.contains('parSMURF')]
d38.columns = d38.columns.str.replace('_parSMURF','')


d = d.set_index('ID',drop = True)
d1= d1.set_index('ID',drop = True) #SNVs.chr5.data.txt


d = d19.corrwith(d38)[1:]
d.name = 'Random'

df = df.join(d)
df.columns = ['DLK1','HBB','PRDM9','Random']
df.index.name = ''

#df.to_latex('figures/identical.tex', index = True,header = True)

In [186]:
df38.corrwith(hg38)


POSITION      1.000000
ISCApath      0.754903
dbVARCount    0.946756
DGVCount      0.987430
dtype: float64

In [26]:
hg19.corrwith(hg38)


POSITION        1.000000
priPhastCons    0.792354
priPhyloP       0.728944
verPhastCons    0.682659
verPhyloP       0.615702
mamPhastCons    0.670038
mamPhyloP       0.510452
DGVCount        1.000000
dtype: float64

#### ISCApath

In [ ]:
# the reason for the correlation equaling NaN
df19['ISCApath'].value_counts()
#for col in df19.iloc[:,1:]:
#    print(df19[col].value_counts())

In [ ]:
hg19['ISCApath'].value_counts()

In [ ]:
hg38['ISCApath'].value_counts()

In [ ]:
x = 23464354
x = x-1 # the class returns a vcf table

In [ ]:
file ='hg19.variants/isca/all_variants_for_nstd75.csv.gz'
df = pd.read_csv(file, sep = ',', skiprows=0)


In [ ]:
#alle.to_csv('input/features/hg38/ISCApath/ISCApath.nstd37_nstd45_nstd75.bed.gz.vartmp',sep = '\t', index = False,header = False,compression = 'gzip')




In [ ]:
# this is the combination of the original files, is produced by the rule variants.snakerule (not the intervals)
file = 'input/features/hg19/ISCApath/ISCApath.nstd37_nstd45_nstd75.bed.gz.vartmp'

df0 = pd.read_csv(file,compression='gzip',sep = '\t',header = None)
df0=df0.sort_values([0,1,2,3]).reset_index(drop=True ) 

df0.head()

In [ ]:
#df[~df[3].isin(df0[3])]

In [ ]:
# this is the interval file 
file = 'input/features/hg19/ISCApath/ISCApath.nstd37_nstd45_nstd75.bed.gz'

df1 = pd.read_csv(file,compression='gzip',sep = '\t',header = None)
df1=df1.sort_values([0,1,2,3]).reset_index(drop=True ) 

df1.head()


In [ ]:
df[(df[0]==x19.chr)&(df[1]<=x)&(df[2]>=x)].shape


In [ ]:
df0[(df0[0]==x19.chr)&(df0[1]<=x)&(df0[2]>=x)].shape


In [ ]:
df1[(df1[0]==x19.chr)&(df1[1]<=x)&(df1[2]>=x)]


In [105]:


df0[(df0[0]==x19.chr)&(df0[1]<=x)&(df0[2]>=x)]#.shape


,0,1,2,3
48554,chr11,4621644,5179426,nsv1054847
48622,chr11,4940287,5212591,esv2760169
48696,chr11,5004969,5193183,nsv553219
48709,chr11,5081245,5227774,esv3891917
48730,chr11,5117502,5186159,nsv553224
48732,chr11,5144422,5175233,esv3625208
48733,chr11,5154246,5248100,dgv1041n100


#### DGVCount

In [116]:
file = 'input/features/hg19/DGVCount/DGVCount.all.bed.gz.dgv'

df2 = pd.read_csv(file,sep = '\t', compression='gzip',header = None)
df2 = df2[~df2[0].str.contains('chrchr',na=False)]

In [123]:
df2[(df2[0]==x19.chr)&(df2[1].astype(int)<=x)&(df2[2].astype(int)>=x)]


,0,1,2,3,4,5
181259,chr5,18907625,26650724,OTHER,inversion,nsv7369
181551,chr5,21523226,29424617,OTHER,inversion,esv3380329
181648,chr5,22419598,24131279,CNV,gain,esv2762506
181710,chr5,23449711,23484372,CNV,gain,nsv597543


In [111]:
alle[(alle[0]==x19.chr)&(alle[1]<=x)&(alle[2]>=x)]


,0,1,2,3,4
3329,chr11,5159347,5167596,7,"nsv1054847, esv2760169, nsv553219, esv3891917,..."


In [79]:
df2[(df2[0]==x19.chr)&(df2[1]<=x)&(df2[2]>=x)]


,0,1,2,3
32828,chr11,4961517,5233820,esv2760169
32873,chr11,5026199,5214412,nsv553219
32896,chr11,5138732,5207388,nsv553224


#### Filter for pathogenic changes the number for ISCAPath

In [ ]:
# these are the original files, must filter for pathogenic
#files = ['nstd75.GRCh38.variant_call.tsv.gz','nstd45.GRCh38.variant_call.tsv.gz','nstd37.GRCh38.variant_call.tsv.gz']
files = ['nstd75.GRCh37.variant_call.tsv.gz','nstd45.GRCh37.variant_call.tsv.gz','nstd37.GRCh37.variant_call.tsv.gz']

cols = ['chr','inner_start','inner_stop','#variant_call_accession']

alle = pd.DataFrame(columns = cols)
st = 'Pathogenic|pathogenic'
#st = 'Pathogenic'

for file in files:
    
    df = pd.read_csv('delete/'+file,compression='gzip',sep = '\t', skiprows =1)
    df = df[~df['clinical_significance'].str.contains(st,na = False)]
    df = df[~df['description'].str.contains(st ,na = False)]
    #for col in df.columns:
    #    print(df[col].astype(str).str.contains('Pathogenic|pathogenic',na = False).sum())
    df = df[~df.chr.isna()]
    print(df.shape)
    alle = alle.append(df)
alle = alle.sort_values(cols)
alle = alle[cols].reset_index(drop = True)
alle.columns = [0,1,2,3]
alle[0]='chr'+alle[0].astype(str)
alle[1]=alle[1].astype(int)-1
alle[2]=alle[2].astype(int)


# when filtered für both pathogenic and pathogeniP 'alle' equals 'df0'
#(df0==alle).sum()
alle.shape

In [ ]:
alle[(alle[0]==x19.chr)&(alle[1]<=x)&(alle[2]>=x)]#.shape


In [ ]:
files = ['nstd75.GRCh37.variant_call.tsv.gz','nstd45.GRCh37.variant_call.tsv.gz','nstd37.GRCh37.variant_call.tsv.gz']
files = ['nstd45.GRCh37.variant_call.tsv.gz','nstd37.GRCh37.variant_call.tsv.gz']

x = 101118632
st = 'pathogenic'

for file in files:
    
    alle = pd.read_csv('delete/'+file,compression='gzip',sep = '\t', skiprows =1)
    alle.to_csv('delete/test_'+file,compression='gzip',sep = ',',index = None)
    alle = alle.sort_values(cols)
    alle = alle[~alle.chr.isna()]
    alle = alle[~alle['clinical_significance'].str.contains(st,na = False)]
    alle = alle[~alle['description'].str.contains(st ,na = False)]

    #display(alle.head())
    alle = alle[cols].reset_index(drop = True)
    alle.columns = [0,1,2,3]
    
    alle[0]='chr'+alle[0].astype(str)
    alle[1]=alle[1].astype(int)-1
    alle[2]=alle[2].astype(int)
    
    display(alle[(alle[0]==x19.chr)&(alle[1]<=x)&(alle[2]>=x)])
    


In [ ]:
alle.head(10000)


In [ ]:
regions = [{'hg19':['chr5',23464354,23570942],'hg38':['chr5',23464245,23570833]},
                    {'hg19':['chr11',5167199,5327798],'hg38':['chr11',5145969,5306568]},
                    {'hg19':['chr14',101118632,101242576],'hg38':['chr14',100652295,100776239]},
                   {'hg19':['chr14',101118632,201242576],'hg38':['chr14',100652295,200776239]}]
a = 23464354
b = 23570942
df[(df[0]=='chr5')&(df[1]>=a)&(df[2]<=b)]

In [ ]:
# these are the original files, must filter for pathogenic
#files = ['nstd75.GRCh38.variant_call.tsv.gz','nstd45.GRCh38.variant_call.tsv.gz','nstd37.GRCh38.variant_call.tsv.gz']
files = ['all_variants_for_nstd75.csv.gz','all_variants_for_nstd37.csv.gz','all_variants_for_nstd45.csv.gz']

cols = ['Chromosome','Inner Start','Inner End','Variant ID']

alle = pd.DataFrame(columns = cols)
st = 'Pathogenic|pathogenic'
#st = 'Pathogenic'

for file in files:
    
    df = pd.read_csv('hg19.variants/isca/'+file,compression='gzip',sep = ',')

    df = df[df['Assembly Name']=='GRCh38 (hg38)'][cols]
    print(df.shape)
    alle = alle.append(df)
alle = alle.sort_values(cols)
alle = alle[cols].reset_index(drop = True)
alle.columns = [0,1,2,3]
alle[0]='chr'+alle[0].astype(str)
alle[1]=alle[1].astype(int)-1
alle[2]=alle[2].astype(int)


alle.to_csv('input/features/hg38/ISCApath/ISCApath.nstd37_nstd45_nstd75.bed.gz.vartmp',sep = '\t', index = False,header = False,compression = 'gzip')



In [ ]:
df = pd.read_csv('input/features/hg38/ISCApath/ISCApath.nstd37_nstd45_nstd75.bed.gz.vartmp',sep = '\t',compression = 'gzip')

df.shape


In [ ]:
regions = [{'hg19':['chr5',23464354,23570942],'hg38':['chr5',23464245,23570833]},
                {'hg19':['chr11',5167199,5327798],'hg38':['chr11',5145969,5306568]},
                {'hg19':['chr14',101118632,101242576],'hg38':['chr14',100652295,100776239]}]
di = regions[2]

for d in ['hg19','hg38']:
    start = di[d][1]
    end = di[d][2]+1
    df = pd.DataFrame(index =range(start,end))
    df['#CHROM'] = di[d][0]
    df['POS'] =  range(start,end)

    df[['ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO']]=[".","T","A",".","PASS","."]
    df = df.T.reset_index().T
    df.columns = ['##fileformat=VCFv4.1']+ ['']*(df.shape[1]-1)
    name = di[d][0]+'.'+'pos'+d[2:]
    
    #df = df.iloc[:90000]
    df.to_csv('validation/'+name+'.vcf',header = True,index = False, sep = '\t')
# bgzip 'validation/'+name+'.vcf'
# tabix 'validation/'+name+'.vcf'
# sbatch_drmaa_snakemake -c cluster.yml -f 'validation/'+name+'.vcf' -j20
#shell('bgzip validation/'+name+'.vcf; tabix validation/'+name+'.vcf;')





In [ ]:
df_all = pd.DataFrame()
for i in list(range(1,23))+['X','Y'][:2]:
    gerp = pd.read_csv('input/features/hg19/GerpRS/hg19_chr'+str(i)+'_elems.txt', sep='\t', header = None)
    gerp[5]='chr'+str(i)
    df_all = df_all.append(gerp[[5,0,1,3,4]])
df_all.to_csv('input/features/hg19/GerpRS/GerpRS.all.bed.gz', sep = '\t',index = False,header=False,compression='gzip')




In [ ]:
start = x38.start
end = start+10

file = 'input/features/hg38/verPhyloP/chr11.phyloP100way.bw'

bb = bw.open(file)
v = bb.values('chr11', di['hg38'][1],di['hg38'][2]) 
file = 'input/features/hg38/verPhyloP/chr11_phyloP100way.bw'
bb = bw.open(file)
v_ = bb.values('chr11', di['hg38'][1],di['hg38'][2]) 

def interval(df,start,end):
    return df[(df.POSITION>=start)&(df.POSITION<end)]


In [ ]:
# Conversation hg19:df19=1
# hg38 (df38) korreliert schlecht mit cadd38. Welche Daten nutzt cadd? für 19 ist die Korr 1
# Unstimmigkeit Encode VCF unf Originaldateien, hg38
# chr5:23464425-23464435
l = glob('input/features/hg38/EncH3K4Me3/*.bigWig')
for file in l:

    bb = bw.open(file)
    print(file)
    v = bb.values('chr5', 23464425,23464435) 
    print(v)
df38[hg38['EncH3K4Me3']!=df38['EncH3K4Me3']]#.head(100)


In [ ]:
df38['POSITION']

In [ ]:
hg38[hg38['EncH3K4Me3']!=df38['EncH3K4Me3']]#.head(100)

In [ ]:
# TEST PRIFASTCONS

hg38[hg38['priPhastCons']!=df38['priPhastCons']].head(10)#['EncH3K4Me3']
df38[hg38['priPhastCons']!=df38['priPhastCons']].head(10)#['EncH3K4Me3']

file = 'input/features/hg38/priPhastCons/hg38_phastCons17way.bw'
bb = bw.open(file)
v = bb.values('chr5', 23464245,23464255) 